In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
import re
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter
animal_shelter = AnimalShelter()

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU123"
shelter = AnimalShelter()

df = pd.DataFrame.from_records(shelter.read({}))

df.drop(columns=['_id'], inplace=True, errors='ignore')

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Load Logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center([
        html.Img(src=f"data:image/png;base64,{encoded_image.decode()}", style={'width': '200px'}),
        html.H3("Dashboard by Greyson - 02/22/25")  # Unique Identifier
    ]),

    html.Center(html.B(html.H1("CS-340 Dashboard"))),
    html.Hr(),

    html.Center(html.Div([
        html.Label('Select Rescue Type:'),
        dcc.Dropdown(
            id='dropdown',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue/Individual Tracking', 'value': 'disaster'}
            ],
            placeholder="Select a rescue type"
        )
    ], style={'width': '50%'})),

    html.Br(),

    html.Div([
        dcc.Input(
            id='global-search',
            type='text',
            placeholder='Search all columns...',
            debounce=True,
            style={'width': '100%', 'padding': '10px', 'fontSize': '16px'}
        )
    ]),

    html.Br(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i} for i in ["name", "breed", "animal_type", "sex_upon_outcome", "age_upon_outcome_in_weeks"]
        ],
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '0px', 'maxWidth': '180px',
            'whiteSpace': 'normal',
            'textAlign': 'left'
        },
        filter_action="native",  # Optional: Keep if you want per-column filters too
        page_action="native",
        page_current=0,
        page_size=10
    )
]),

    html.Br(),
    html.Hr(),

    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                 html.Div(id='graph-id', className='col s12 m6'),
                 html.Div(id='map-id', className='col s12 m6')
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('dropdown', 'value')]
)
def update_dashboard(filter_type):
    df = pd.DataFrame.from_records(shelter.read({}))  # Get data
    
    # Convert columns to appropriate formats
    df["age_upon_outcome_in_weeks"] = pd.to_numeric(df["age_upon_outcome_in_weeks"], errors='coerce')
    df["breed"] = df["breed"].str.lower()  # Ensure case-insensitive matching

    # Convert any datetime columns to strings
    if 'datetime' in df.columns:
        df['datetime'] = df['datetime'].astype(str)

    # Drop the _id column if it's present
    df.drop(columns=['_id'], inplace=True, errors='ignore')

    # Apply filter based on dropdown selection
    if filter_type == 'water':
        query = df[(df["breed"].isin(["labrador retriever mix", "chesapeake bay retriever", "newfoundland"])) & 
                   (df["sex_upon_outcome"] == "Intact Female") & 
                   (df["age_upon_outcome_in_weeks"].between(26, 156))]
    elif filter_type == 'mountain':
        query = df[(df["breed"].isin(["german shepherd", "alaskan malamute", "old english sheepdog", "siberian husky", "rottweiler"])) & 
                   (df["sex_upon_outcome"] == "Intact Male") & 
                   (df["age_upon_outcome_in_weeks"].between(26, 156))]
    elif filter_type == 'disaster':
        query = df[(df["breed"].isin(["doberman pinscher", "german shepherd", "golden retriever", "bloodhound", "rottweiler"])) & 
                   (df["sex_upon_outcome"] == "Intact Male") & 
                   (df["age_upon_outcome_in_weeks"].between(20, 300))]
    else:
        query = df

    # Handle empty query
    if query.empty:
        return []

    return query.to_dict('records')


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        return []
    dff = pd.DataFrame(viewData)
    return [
        dcc.Graph(
            figure=px.pie(dff, names='breed', title='Preferred Animals')
        )
    ]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'backgroundColor': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

app.run_server(debug=True)


In [ ]:
dash_table.DataTable(
    id='datatable-id',
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
    data=df.to_dict('records')
)

In [ ]:
df.iloc[0]

In [ ]:
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": "Animal Name", "id": "name", "deletable": False, "selectable": True},
            {"name": "Animal ID", "id": "animal_id", "deletable": False, "selectable": True},
            {"name": "Animal Age", "id": "age_upon_outcome", "deletable": False, "selectable": True},
            {"name": "Most Recent Event", "id": "outcome_type", "deletable": False, "selectable": True}
        ]